In [ ]:
import random as r
import itertools

In [ ]:
class Bacteria:
    def __init__(self, R=-1, G=-1, B=-1, target=[96, 96, 159]):
        if R == G == B == -1:
            self.__R = r.randint(0, 256)
            self.__G = r.randint(0, 256)
            self.__B = r.randint(0, 256)
        else:
            self.__R = R
            self.__G = G
            self.__B = B
        
        self.__RT = target[0]
        self.__GT = target[1]
        self.__BT = target[2]
            
        self.__pm = 0.4
            
        
    def _evaluete(self):
        """Считаем насколько данная особь соответствует таргету"""
        return ((self.__RT - self.__R) ** 2 + (self.__GT - self.__G) ** 2 + (self.__BT - self.__B) ** 2) ** .5
    
    
    def __add__(self, other):
        """Перегрузка будет скрещивать 2 объекта класса Bacteria и возвращать 2х особей потомка с помощью
        одноточечного кроссовера"""
        
        gen = r.randint(0, 2)
        
        genom_1 = self()
        genom_2 = other()
        
        gen_1 = genom_1.pop(gen)
        gen_2 = genom_2.pop(gen)
        
        new_gen = int("{0:b}".format(gen_1).zfill(4)[:4] + "{0:b}".format(gen_2).zfill(8)[4:8], 2) + r.randint(0,1)
        combinations = [list(comb) for comb in itertools.product(genom_1, genom_2)]
        
        
        mutated_bact = []
        for comb in combinations:
            comb_c = comb.copy()
            if r.random() < self.__pm:
                mutated_index = r.randint(0, 1)
                mutated = "{0:b}".format(comb[mutated_index]).zfill(8)
                
                bit_indices = r.sample(range(len(mutated)), 3)
                inverted_num = ''.join([str((int(bit, 2) + 1) % 2) if i in bit_indices else bit for i, bit in enumerate(mutated)]) 
                
                comb_c[mutated_index] = int(inverted_num, 2) + r.randint(0,1)
                
            mutated_bact.append(comb_c)
        mutated_bact = combinations
        final_out = []
        for comb in mutated_bact:
            comb_c = comb.copy()
            comb_c.insert(gen, new_gen)
            
            final_out.append(comb_c)
        
        return [Bacteria(*i) for i in final_out]
    
        
    # Перегружаем больше, возвращаем меньше, да я
    def __gt__(self, other):
        return self._evaluete() < other._evaluete()
    
    
    def __call__(self):
        return [self.__R, self.__G, self.__B]
    
    
    def __eq__(self, other):
        return self.__R == other.__R and self.__G == other.__G and self.__B == other.__B

In [ ]:
# у нас будет турнирная селекция, 
def binary_tournament_selection(population):
    selected_parents = []
    
    for _ in range(len(population)):
        candidate_1, candidate_2 = r.sample(population, 2)
        
        if  candidate_1 > candidate_2:
            if candidate_1 not in selected_parents:
                selected_parents.append(candidate_1)
        else:
            if candidate_2 not in selected_parents:
                selected_parents.append(candidate_2)
    return selected_parents

In [ ]:
def top_fitness(population):
    x = [x._evaluete() for x in population]
    return sorted(x)[:3]

In [ ]:
x = Bacteria(1, 1, 1)
y = Bacteria(1, 1, 1)

In [ ]:
start_pop = [Bacteria() for i in range(200)]
len_start_prop = len(start_pop)
selected = binary_tournament_selection(start_pop)
target = Bacteria(96, 96, 159)

counter = 0
flag = False
while not flag:
    counter += 1
    for prop in selected:
        if prop == target:
            print(prop(), counter)
            flag = True
            break
    new_pop = []
    for i in range(0, len(selected)):
        if 0.4 > r.random():
            new_pop.extend(selected[i-1] + selected[i])
        else:
            new_pop.append(selected[i-1])
            new_pop.append(selected[i])

    new_pop.sort(key=Bacteria._evaluete)
    # selected = binary_tournament_selection(new_pop)
    selected = new_pop[:len_start_prop]


KeyboardInterrupt: 